In [ ]:
counter = {}

for p in range(0, 1, 100): # p è la confidenza  
  counter[p] = 0
  total_pixels = images.elems()
  for image in images:
    for pixel in image:
      mean, std = pixel['prediction']
      y = pixel['true_value']
      lower_bound, upper_bound = compute_confidence_interval(p, mean, std, sample_size = 1)
  
      if y <= mean + upper_bound and y >= mean - lower_bound:
        counter[p] += 1

  p^ = counter[p] / total_pixels


# Un sistema perfettamente calibrato avrà p^ = p. Ovviamente non sarà così,
# e quindi ci sarà un ondulamente del valore p^ rispetto alla perfezione
# di p. L'area intermedia fra queste due curve equivale al nostro errore.

# Attenzione: come calcolare l'intervallo di confidenza è un problema.
# Ogni distribuzione ha il suo modo. La normale ha per esempio una tabella 
# di riferimento, questo perchè calcolare queste cose non è proprio semplice.
# Equivale all'integrale della PDF.

  

In [22]:
for i in np.arange(0, 0.99, 0.01):
  print(i+0.01)

0.01
0.02
0.03
0.04
0.05
0.060000000000000005
0.06999999999999999
0.08
0.09
0.09999999999999999
0.11
0.12
0.13
0.14
0.15000000000000002
0.16
0.17
0.18000000000000002
0.19
0.2
0.21000000000000002
0.22
0.23
0.24000000000000002
0.25
0.26
0.27
0.28
0.29000000000000004
0.3
0.31
0.32
0.33
0.34
0.35000000000000003
0.36000000000000004
0.37
0.38
0.39
0.4
0.41000000000000003
0.42000000000000004
0.43
0.44
0.45
0.46
0.47000000000000003
0.48000000000000004
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.5700000000000001
0.5800000000000001
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.6900000000000001
0.7000000000000001
0.7100000000000001
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.8200000000000001
0.8300000000000001
0.8400000000000001
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.9400000000000001
0.9500000000000001
0.9600000000000001
0.97
0.98
0.99


In [2]:
import numpy as np
a = np.array([[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]])

b = np.array([[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]])

c = np.array([[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]])

In [9]:
np.sum((b >= a) * (b <= c))

9

In [ ]:
def compute_ause(input_batch, result_batch):
    """Compute the Area Under the Sparsification Error (AUSE)."""
    ause = []
    input_batch = input_batch.cpu()
    result_batch = {k: v.cpu() for k, v in result_batch.items()}
    for instance_id in range(input_batch.shape[0]):
        input_instance = input_batch[instance_id][0]
        mean_result = result_batch['mean'][instance_id][0]
        var_result = result_batch['var'][instance_id][0]
        
        # Compute sparsification curves for the predicted depth map
        error = (input_instance - mean_result).pow(2) # RMSE -> SE (without mean and root)
        sparsification_prediction = sparsification(input_instance,
                                                   mean_result,
                                                   var_result)
        sparsification_oracle = sparsification(input_instance, 
                                               mean_result, 
                                               error)
        # Calculate the error difference between the sparsification curves
        sparsification_errors = sparsification_oracle - sparsification_prediction
        # Compute the AUSE by integrating the absolute values of the error differences
        ause = np.trapz(np.abs(sparsification_errors), np.arange(sparsification_errors.shape[0]))
    return ause.mean()


